<a href="https://www.kaggle.com/code/darvack/transformer-paper-regression-mix?scriptVersionId=132792290" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
random.seed(42)
np.random.seed(42)

Here, we have loaded the data and set Furan as the label. At first, we have used KFold Round-Robin for testing our model so that every data points gets tested 1 time.

In [2]:
ds_A = pd.read_csv("/kaggle/input/transformer/DatasetA.csv")
ds_B = pd.read_csv("/kaggle/input/transformer/DatasetB.csv")

ds_A_new = ds_A.drop(set(ds_A.columns) - set(ds_B.columns), axis=1)
ds_B_new = ds_B[ds_A_new.columns]

ds_mix = pd.concat([ds_A_new, ds_B_new]).sample(frac=1, random_state=42).reset_index(drop=True)
y_mix = ds_mix['Furan']
X_mix = ds_mix.drop(["Furan", "HI"], axis = 1)

y_stratify = pd.DataFrame(y_mix.copy())
bins = [-1, 0.1, 1, 100]
labels = [0, 1, 2]
y_stratify['Class'] = pd.cut(y_stratify['Furan'], bins=bins, labels=labels)
y_stratify = y_stratify.drop(["Furan"], axis = 1) 

In [3]:
#ds_mix.hist(bins=50, figsize=(20,15))

The code below performs feature selection via step-wise regression or SequentialFeatureSelector. "n_features_to_select" is a hyperparameter that defines how many feature we'd like to keep which needs to be fine tuned. In our experiment, 5 or 7 features to keep, results in better models. Here we used 7.

In [4]:
'''from sklearn.feature_selection import SequentialFeatureSelector
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

for i in range(1,9):
    rf_reg = RandomForestRegressor(min_samples_split=12, min_samples_leaf=1, random_state=42)
    
    xgb_reg = XGBRegressor(learning_rate=0.01, n_estimators=300, max_depth=3, subsample=0.7)

    sfs = SequentialFeatureSelector(rf_reg,
                                    n_features_to_select=i,
                                    direction='forward',
                                    scoring='neg_mean_squared_error',
                                    cv=4)

    sfs.fit(X_mix, y_mix)

    # Print the selected features
    print("Selected features" + f"{i}:", sfs.get_feature_names_out())
'''

useful_features1 = ['IFT']
useful_features2 = ['Methane', 'IFT']
useful_features3 = ['Methane', 'Ethane', 'IFT']
useful_features4 = ['Methane', 'Ethane', 'Water', 'IFT']
useful_features5 = ['Methane', 'Ethylene', 'Ethane', 'BDV', 'IFT']
useful_features6 = ['Methane', 'Ethylene', 'Ethane', 'Water', 'BDV', 'IFT']
useful_features7 = ['H2', 'Methane', 'Acetylene', 'Ethylene', 'Ethane', 'BDV', 'IFT']
useful_features8 = ['H2', 'Methane', 'Ethylene', 'Ethane', 'Water', 'Acid', 'BDV', 'IFT']

In [5]:
Selected_features1 = ['Acid']
Selected_features2 = ['Ethane', 'Acid']
Selected_features3 = ['Ethane', 'Water', 'Acid']
Selected_features4 = ['Ethane', 'Water', 'Acid', 'IFT']
Selected_features5 = ['Methane', 'Ethane', 'Water', 'Acid', 'IFT']
Selected_features6 = ['Methane', 'Acetylene', 'Ethane', 'Water', 'Acid', 'IFT']
Selected_features7 = ['Methane', 'Acetylene', 'Ethylene', 'Ethane', 'Water', 'Acid', 'IFT']
Selected_features8 = ['H2', 'Methane', 'Acetylene', 'Ethylene', 'Ethane', 'Water', 'Acid', 'IFT']

In [6]:
'''Selected_features1= ['Acid']
Selected_features2= ['Acid', 'IFT']
Selected_features3= ['Water', 'Acid', 'IFT']
Selected_features4= ['Water', 'Acid', 'BDV', 'IFT']
Selected_features5= ['Methane', 'Water', 'Acid', 'BDV', 'IFT']
Selected_features6= ['Methane', 'Acetylene', 'Water', 'Acid', 'BDV', 'IFT']
Selected_features7= ['Methane', 'Acetylene', 'Ethane', 'Water', 'Acid', 'BDV', 'IFT']
Selected_features8= ['H2', 'Methane', 'Acetylene', 'Ethane', 'Water', 'Acid', 'BDV', 'IFT']'''

"Selected_features1= ['Acid']\nSelected_features2= ['Acid', 'IFT']\nSelected_features3= ['Water', 'Acid', 'IFT']\nSelected_features4= ['Water', 'Acid', 'BDV', 'IFT']\nSelected_features5= ['Methane', 'Water', 'Acid', 'BDV', 'IFT']\nSelected_features6= ['Methane', 'Acetylene', 'Water', 'Acid', 'BDV', 'IFT']\nSelected_features7= ['Methane', 'Acetylene', 'Ethane', 'Water', 'Acid', 'BDV', 'IFT']\nSelected_features8= ['H2', 'Methane', 'Acetylene', 'Ethane', 'Water', 'Acid', 'BDV', 'IFT']"

# First case: Using 4 different folds for test set (Round-Robin)

In [7]:
'''from sklearn.ensemble import RandomForestRegressor
#from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
#from sklearn.linear_model import Ridge
import lightgbm as lgb
#from sklearn.tree import DecisionTreeRegressor

rf_reg = RandomForestRegressor(n_jobs = -1, max_depth = 50)
# svm_reg = SVR(kernel='linear')
knn_reg = KNeighborsRegressor(n_neighbors=3)
xgb_reg = XGBRegressor(learning_rate=0.01, n_estimators=300, max_depth=3, subsample=0.7)
ada_reg = AdaBoostRegressor(n_estimators=50, learning_rate=0.003)
#lr_reg = LinearRegression()
cat_reg = CatBoostRegressor(iterations=500, learning_rate=0.1, verbose = 0,
                           depth=6, loss_function='RMSE', random_seed=11)
lgb_reg = lgb.LGBMRegressor()
#el_reg = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=0)'''

"from sklearn.ensemble import RandomForestRegressor\n#from sklearn.linear_model import LinearRegression\nfrom sklearn.svm import SVR\nfrom sklearn.neighbors import KNeighborsRegressor\nfrom xgboost import XGBRegressor\nfrom sklearn.ensemble import AdaBoostRegressor\nfrom catboost import CatBoostRegressor\n#from sklearn.linear_model import Ridge\nimport lightgbm as lgb\n#from sklearn.tree import DecisionTreeRegressor\n\nrf_reg = RandomForestRegressor(n_jobs = -1, max_depth = 50)\n# svm_reg = SVR(kernel='linear')\nknn_reg = KNeighborsRegressor(n_neighbors=3)\nxgb_reg = XGBRegressor(learning_rate=0.01, n_estimators=300, max_depth=3, subsample=0.7)\nada_reg = AdaBoostRegressor(n_estimators=50, learning_rate=0.003)\n#lr_reg = LinearRegression()\ncat_reg = CatBoostRegressor(iterations=500, learning_rate=0.1, verbose = 0,\n                           depth=6, loss_function='RMSE', random_seed=11)\nlgb_reg = lgb.LGBMRegressor()\n#el_reg = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=0)"

**without any feature selection**

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
#from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
#from sklearn.linear_model import Ridge
import lightgbm as lgb
#from sklearn.tree import DecisionTreeRegressor

rf_reg = RandomForestRegressor(min_samples_split=12, min_samples_leaf=1, random_state=42)
knn_reg = KNeighborsRegressor(n_neighbors=7)
xgb_reg = XGBRegressor(random_state=42, learning_rate=0.01, n_estimators=500, max_depth=3, subsample=0.95)
ada_reg = AdaBoostRegressor(random_state=42, n_estimators=50, learning_rate=0.003)
cat_reg = CatBoostRegressor(iterations=100, learning_rate=0.3, verbose=0, depth=6, loss_function='RMSE', random_seed=42)
lgb_reg = lgb.LGBMRegressor(boosting_type='dart', n_estimators=150, min_child_samples=15, reg_alpha=0.01)

regressors = [ada_reg,knn_reg,xgb_reg,rf_reg,cat_reg,lgb_reg]

for reg in regressors:
    scores = cross_val_score(reg, X_mix, y_mix, cv=4, scoring='neg_mean_squared_error')
    mse_scores = -scores
    print(reg.__class__.__name__ + " for dataset A:")
    print("MSE Scores:", mse_scores)
    print("Mean MSE:", mse_scores.mean())
    print()

AdaBoostRegressor for dataset A:
MSE Scores: [1.50119952 0.66517946 0.60400273 0.84700903]
Mean MSE: 0.9043476830654802

KNeighborsRegressor for dataset A:
MSE Scores: [2.76597775 0.83790695 0.62791931 1.16359512]
Mean MSE: 1.3488497833323927

XGBRegressor for dataset A:
MSE Scores: [1.63540435 0.53575221 0.45364421 0.70753409]
Mean MSE: 0.8330837118065768

RandomForestRegressor for dataset A:
MSE Scores: [1.42238328 0.51413721 0.52022166 0.74549698]
Mean MSE: 0.8005597832647249

CatBoostRegressor for dataset A:
MSE Scores: [1.3724171  0.41031634 0.59708145 0.82862767]
Mean MSE: 0.8021106428151465

LGBMRegressor for dataset A:
MSE Scores: [1.60394279 0.5505089  0.49078859 0.85054179]
Mean MSE: 0.8739455168889977



**With feature selection**

In [9]:
rf_reg = RandomForestRegressor(min_samples_split=12, min_samples_leaf=1, random_state=42)
knn_reg = KNeighborsRegressor(n_neighbors=7)
xgb_reg = XGBRegressor(random_state=42, learning_rate=0.01, n_estimators=500, max_depth=3, subsample=0.95)
ada_reg = AdaBoostRegressor(random_state=42, n_estimators=50, learning_rate=0.003)
cat_reg = CatBoostRegressor(iterations=100, learning_rate=0.3, verbose=0, depth=6, loss_function='RMSE', random_seed=42)
lgb_reg = lgb.LGBMRegressor(boosting_type='dart', n_estimators=150, min_child_samples=15, reg_alpha=0.01)

regressors = [ada_reg,knn_reg,xgb_reg,rf_reg,cat_reg,lgb_reg]

for reg in regressors:
    scores = cross_val_score(reg, X_mix[Selected_features7], y_mix, cv=4, scoring='neg_mean_squared_error')
    mse_scores = -scores
    print(reg.__class__.__name__ + " for dataset A:")
    print("MSE Scores:", mse_scores)
    print("Mean MSE:", mse_scores.mean())
    print()

AdaBoostRegressor for dataset A:
MSE Scores: [1.46588666 0.66840179 0.58160909 0.86588001]
Mean MSE: 0.8954443871493902

KNeighborsRegressor for dataset A:
MSE Scores: [2.48213958 0.64038244 0.71939994 1.27151644]
Mean MSE: 1.2783596000740842

XGBRegressor for dataset A:
MSE Scores: [1.5586981  0.53299672 0.45711862 0.68525145]
Mean MSE: 0.8085162254002447

RandomForestRegressor for dataset A:
MSE Scores: [1.39725749 0.47736157 0.52318533 0.73978485]
Mean MSE: 0.7843973096603026

CatBoostRegressor for dataset A:
MSE Scores: [1.6434233  0.48307716 0.60181894 0.71622408]
Mean MSE: 0.8611358691927917

LGBMRegressor for dataset A:
MSE Scores: [1.65718146 0.4933465  0.54443024 0.78616408]
Mean MSE: 0.8702805706725256



# Blending model

In [10]:
'''train_valid_mix, test_mix = train_test_split(ds_mix, test_size = 0.25, random_state = 42, stratify=y_stratify)

train_mix, valid_mix = train_test_split(train_valid_mix, test_size = 0.15, random_state = 42, stratify=y_stratify.loc[train_valid_mix.index])

y_train = train_mix['Furan']
X_train = train_mix.drop(['Furan', 'HI'], axis=1)

y_test = test_mix['Furan']
X_test = test_mix.drop(['Furan', 'HI'], axis=1)

y_valid = valid_mix['Furan']
X_valid = valid_mix.drop(['Furan', 'HI'], axis=1)'''

"train_valid_mix, test_mix = train_test_split(ds_mix, test_size = 0.25, random_state = 42, stratify=y_stratify)\n\ntrain_mix, valid_mix = train_test_split(train_valid_mix, test_size = 0.15, random_state = 42, stratify=y_stratify.loc[train_valid_mix.index])\n\ny_train = train_mix['Furan']\nX_train = train_mix.drop(['Furan', 'HI'], axis=1)\n\ny_test = test_mix['Furan']\nX_test = test_mix.drop(['Furan', 'HI'], axis=1)\n\ny_valid = valid_mix['Furan']\nX_valid = valid_mix.drop(['Furan', 'HI'], axis=1)"